In [1]:
cd /content/drive/My Drive/tobigs

/content/drive/My Drive/tobigs


### 데이터 전처리

In [0]:
import numpy as np
import pandas as pd

sample_submission = pd.read_csv("sample_submission.csv")
train = pd.read_csv("train_df.csv")
test = pd.read_csv("test_df.csv")

In [0]:
X = train.iloc[:, 1:].values / 255
y = train.iloc[:, 0].values
test = test.iloc[:, 1:].values / 255

### Dataset, DataLoader

In [0]:
from torch.utils.data import Dataset, DataLoader
import torch #필요한 라이브러리 import 

class TrainDataset(Dataset): #trainset dataloader 정의
    def __init__(self, X, y, transform=None): #데이터셋의 전처리를 해주는 부분
        self.X = X
        self.y = y
        self.transform = transform
    def __len__(self): #데이터셋의 길이. 즉 총 샘플의 수를 적어주는 부분
        return len(self.X)
    def __getitem__(self, idx): #데이터셋에서 특정 1개의 샘플을 가져오는 함수
        X, y = self.X[idx], self.y[idx]
        x2d = X.reshape(28, 28)
        x2d = X.reshape(784)
        return torch.tensor(X).float(), torch.tensor(y)

class TestDataset(Dataset): #위의 학습데이터와 동일한 방식으로 testdataloader 생성
    def __init__(self, X, y=None, transform=None): #데이터셋의 전처리를 해주는 부분
        self.X = X
        self.y = y
        self.transform = transform
    def __len__(self): #데이터셋의 길이. 즉 총 샘플의 수를 적어주는 부분
        return len(self.X)

    def __getitem__(self, idx): #데이터셋에서 특정 1개의 샘플을 가져오는 함수
        X = self.X[idx]
        return torch.tensor(X).float()

In [0]:
traindataset = TrainDataset(X, y)
testdataset = TestDataset(test)

In [0]:
trainloader = DataLoader(traindataset, batch_size=256, shuffle=True, num_workers=2)
testloader = DataLoader(testdataset, batch_size=256, shuffle=False, num_workers=2)

### Model

Model

-Activation : 활성화 함수, 신경망의 출력을 결정하는 식 elu, selu, relu, leakyrelu, sigmoid 를 사용해보았고, loss가 가장 작게 나온 elu를 사용하였습니다 elu는 relu의 특성을 공유하면서, 그래디언트가 죽지 않는다는 장점이 있다고합니다 

-Batch normalization: 각 배치에서 이전 레이어의 activations을 정규화하는 batchnormalization을 진행했고 이동평균 및 이동 분산에 대한 모멘텀을 의미하는 momentum을 따로 설정해주었습니다

-Regularization: overfitting을 막기위해 필요한 규제로 dropout사용했습니다 neural net 중간중간에 넣었습니다! -> 학습을 하면서 weight들이 서로 동조하는 현상 피하기 위해

In [7]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() #총 4개의 layer 설정
        self.fc1 = nn.Linear(784, 512) #첫번재 layer
        self.bn1 = nn.BatchNorm1d(512,momentum=0.2) #batch normalization
        self.fc2 = nn.Linear(512, 256) #두번째 layer
        self.bn2 = nn.BatchNorm1d(256,momentum=0.2)#batch normalization
        self.fc3 = nn.Linear(256, 128)#세번째 layer
        self.bn3 = nn.BatchNorm1d(128,momentum=0.2)
        self.fc4 = nn.Linear(128, 10)#네번째 layer
        self.elu = nn.ELU() #activation function은 elu사용!
        self.dropout1 = torch.nn.Dropout(p=0.5)#dropout
        self.dropout2 = torch.nn.Dropout(p=0.5) #regularization위한 dropout설정

    def forward(self, x):
        x = self.fc1(x) #첫번째 선형결합후
        x = self.fc2(self.bn1(F.relu(x)))#활성화함수relu를 지나고 bactch normalization을 거친후 두번째 layer로 들어가 선형결합
        x = self.dropout1(x) #regularization을 위한 dropout과정을 지나고
        x = self.fc3(self.bn2(self.elu(x))) #다시 활성화함수 relu를 거친후 batchnormalization 후 세번째 layer로 들어간다
        x = self.dropout2(x) #두번째 dropout
        x = self.fc4(self.bn3(x)) #또 batchnormalization후 마지막 선형결합
        return x #결과값 리턴받음

net = Net()

net

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.2, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.2, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.2, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=128, out_features=10, bias=True)
  (elu): ELU(alpha=1.0)
  (dropout1): Dropout(p=0.5, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
)



*   Optimization: optimizer로 rmsprop, adam, adagrad 사용해보았는데 사실 거의 비슷하게 나왔는데 가장 loss 작게 나온 adam 선택했습니다! adam은 adagrad와 rmsprop의 학습 파라미터의 개별 원소마다 학습률을 달리 적용하는 기법을 합친것!



In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()#loss는 crossentropy로 확인
optimizer = optim.Adam(net.parameters(), lr=0.0001)# optimizer사용 #learningrate 따로 설정함

In [9]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
print(device)

cuda


In [0]:
model = net.to(device) #모델 적용

-Weight Initialization: 각 층의 가중치(weights)와 편항(bias)등 학습 파라미터는 초기값 설정이 매우 중요하기 때문에,nn.init.kaiming_normal_(m.weight)을 사용했습니다!

In [0]:
def weights_init(m): #weight initialization
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight) #kaiming_normal_이 he_normal이랑 같음
 
model = model.apply(weights_init)#가중치 초기화 함수 모델에 적용

In [18]:
num_epochs = 10 #10번정도 더 돌려볼려고 10쓴거고 사실상 150번정도 돌렸습니다
for epoch in range(num_epochs):  # epoch수만큼 데이터 학습

    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
      #앞서 정의한 trainloader에서 학습데이터를 불러오고
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad() 

        # forward + backward + optimize
        outputs = model(inputs) #미리 정의해둔 모델을 적용한다
        loss = criterion(outputs, labels) #loss를 구하면서, forward로 구한 gradient를 이용해 back propagation을 이용해 가중치를 update해간다
        loss.backward()#backpropagation과정
        optimizer.step() #optimizer을 이용해 loss최소화시키는 최적화 단계 진행

        # print statistics
        running_loss += loss.item()
    print('[%d, %5d] loss: %.9f' %(epoch + 1, i + 1, running_loss / 20)) #epoch당 loss출력
    running_loss = 0.0

print('Finished Training')


[1,   165] loss: 0.005083313
[2,   165] loss: 0.028569715
[3,   165] loss: 0.004760776
[4,   165] loss: 0.003316136
[5,   165] loss: 0.005522966
[6,   165] loss: 0.003453139
[7,   165] loss: 0.003033828
[8,   165] loss: 0.002869583
[9,   165] loss: 0.003166222
[10,   165] loss: 0.006621642
Finished Training


In [19]:
correct = 0
total = 0
preds = []
with torch.no_grad():
    for data in testloader: #미리 정의해둔 testloader로 test데이터 불러온다
        images = data
        images = images.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        for pred in predicted: #만들어둔 모델을 이용해 test에대한 y값을 예측한 prediction data를 만든다
          preds.append(pred.cpu().numpy())
print(preds[:10])

[array(8), array(0), array(5), array(3), array(8), array(1), array(9), array(6), array(6), array(0)]


In [20]:
preds[:10]

[array(8),
 array(0),
 array(5),
 array(3),
 array(8),
 array(1),
 array(9),
 array(6),
 array(6),
 array(0)]

In [21]:
import numpy as np
preds = np.array(preds)
print(preds.shape)

(18000,)


In [22]:
sample_submission["Category"] = pd.Series(preds)
sample_submission.head()

,Id,Category
0,57808,8
1,4960,0
2,35755,5
3,15543,3
4,48968,8


In [0]:
sample_submission.to_csv("torch_submission.csv", index=False)